In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize 
import itertools

#load the data
data = pd.read_feather("./../data/raw_text_DF.feather")
data["category"] = pd.Categorical(data["category"])
print(data.shape)
X = data["text"]
Y = pd.get_dummies(data["category"])
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.1)

(2225, 3)


In [55]:
X_unique = [word_tokenize(text.lower()) for text in train_X]
X_unique = pd.value_counts(list(itertools.chain.from_iterable(X_unique)))
X_unique = list(X_unique.index)

## Tf-idf feature representation

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(vocabulary=X_unique) 
X_train_tfidf = tfidf.fit_transform(train_X)
X_test_tfidf = tfidf.transform(test_X)

Y_train =  np.array(train_Y).argmax(1)
Y_test =  np.array(test_Y).argmax(1)

In [57]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(2002, 33283)
(223, 33283)


## Train the model

In [58]:
from sklearn.naive_bayes import MultinomialNB
import time
start_time = time.time()
clf = MultinomialNB()
clf.fit(X_train_tfidf.todense(), Y_train)
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.25519394874572754 seconds ---


## Train set accuracy

In [59]:
np.sum(clf.predict(X_train_tfidf.todense())==Y_train)/len(Y_train)

0.98001998001998

## Test set accuracy

In [60]:
np.sum(clf.predict(X_test_tfidf.todense())==Y_test)/len(Y_test)

0.9641255605381166